<a href="https://colab.research.google.com/github/MarionMoseby/bigdata_finalproject/blob/main/Annex_I_Xray_Images_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This jupyter notebook shall serve as accompanying material to this repositories' README, a report for the “Big Data Engineering” subject at UPM’s Master in Computational Biology. It is thus only intended as a recopilation of used code; for the full discussion, please refer to the README.

**DEPENDENCIES INSTALLATION**

In [ ]:
# First, we install jdk8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
# And set the environment variable 'JAVA_HOME'.
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [ ]:
# We will also import sys (for conda) and set the python version to 3.7 (for orca)
import sys; python_version = f"3.7.10"

In [ ]:
# We will install Miniconda to manage package instalations
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# Update it
!conda install --channel defaults conda python=$python_version --yes
!conda update --channel defaults --all --yes

# And append it to the sys.path
_ = (sys.path
        .append(f"/usr/local/lib/python3.7/site-packages"))

os.environ['PYTHONHOME']="/usr/local"

--2022-02-10 21:40:56--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2022-02-10 21:40:56--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh.2’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   234MB/s    in 0.2s    

2022-02-10 21:40:57 (234 MB/s) - ‘Miniconda3-4.5.4-Linux-x8

In [ ]:
# For our ML processing, we will be using the latest pre-release version of BigDL Orca 
# Installing BigDL Orca from pip will automatically install pyspark, bigdl, and their dependencies.
!pip install --pre --upgrade bigdl-orca

In [ ]:
# Sone other dependencies have to be installed manually
!pip install torch==1.7.1 torchvision==0.8.2
!pip install six cloudpickle
!pip install jep==3.9.0

In [ ]:
# Since we are working on colab, we will mount google drive into our system
# in order to work with the necessary databases.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**DATA PRE-PROCESSING**

In [ ]:
# We will unzip the files (once only, thus commented) to our cloud storage and then
#!unzip /content/drive/MyDrive/Colab_Notebooks/archive.zip -d /content/drive/MyDrive/Colab_Notebooks/

# define the path to the different folders:
test_path = '/content/drive/MyDrive/Colab_Notebooks/chest_xray/test'
train_path = '/content/drive/MyDrive/Colab_Notebooks/chest_xray/train'
validation_path = '/content/drive/MyDrive/Colab_Notebooks/chest_xray/val'

In [ ]:
# For the processing of the images, we are going to be using pytorch
# a ML Library originally developed by facebook

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as T

In [ ]:
# First, we define a function that transforms (pre-processes) the images
def custom_transform(sample):
  transformer = torchvision.transforms.Compose([T.CenterCrop(size=(299, 299)), T.ToTensor(), T.RandomHorizontalFlip(p=0.5), 
                                  T.ColorJitter(brightness=0.5, hue=0), 
                                  T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
  return transformer(sample["image"]), sample["label"]

In [ ]:
# Now, we will use some torch built-in functions to create a custom dataset 
# that we will feed to orca in order to create the model
from torch.utils.data import Dataset, DataLoader
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image

class CustomDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        normal_names = ["NORMAL/" + f for f in listdir(join(root_dir, "NORMAL")) if isfile(join(root_dir, "NORMAL", f))]
        labels_normal = [0]*len(normal_names)
        pneumonia_names = ["PNEUMONIA/" + f for f in listdir(join(root_dir, "PNEUMONIA")) if isfile(join(root_dir, "PNEUMONIA", f))]
        labels_pneumonia = [1]*len(pneumonia_names)
        self.labels = labels_normal
        self.labels.extend(labels_pneumonia)
        self.labels = np.asarray(self.labels, dtype=np.float32)
        # labelling done

        self.filenames = normal_names
        self.filenames.extend(pneumonia_names)

        

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.filenames[idx])
        image = Image.open(img_name).convert("RGB")
        #print(image.shape)
        label = torch.Tensor([self.labels[idx]])
        sample = {"image": image, "label": label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
# We can now re-define the data to be of our 'CustomDataset' class

train_data = CustomDataset(train_path, transform=custom_transform)
val_data = CustomDataset(validation_path, transform=custom_transform)
test_data = CustomDataset(test_path, transform=custom_transform)

In [ ]:
# We load the data into pytorch
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
print('Number of images for training: ', len(train_data))
print('Number of images for testing: ', len(test_data))
print('Number of images for validation: ', len(val_data))

Number of images for training:  5216
Number of images for testing:  624
Number of images for validation:  16


**INTEGRATED NETWORK STACKING**

In [ ]:
# We first import yet some more modules
# In this case, the ones we will use for ML
from __future__ import print_function
import os
import argparse

from bigdl.orca import init_orca_context, stop_orca_context
from bigdl.orca import OrcaContext

In [ ]:
# This will display terminal's stdout and stderr in the Jupyter notebook,
# and is recommended when running BigDL in Jupyter, as it makes troubleshooting easier
OrcaContext.log_output = True 

cluster_mode = "local"

if cluster_mode == "local": # run in local mode
    init_orca_context(cores=1, memory="8g") 
elif cluster_mode == "k8s": # run on K8s cluster
    init_orca_context(cluster_mode="k8s", num_nodes=2, cores=4) 
elif cluster_mode == "yarn": # run on Hadoop YARN cluster
    init_orca_context(
        cluster_mode="yarn-client", cores=4, num_nodes=2, memory="2g",
        driver_memory="10g", driver_cores=1,
        conf={"spark.rpc.message.maxSize": "1024",
              "spark.task.maxFailures": "1",
              "spark.driver.extraJavaOptions": "-Dbigdl.failure.retryTimes=1"}) 

Initializing orca context
Current pyspark location is : /usr/local/lib/python3.7/site-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /usr/local/lib/python3.7/site-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_2.4.6-0.14.0-SNAPSHOT-jar-with-dependencies.jar:/usr/local/lib/python3.7/site-packages/bigdl/share/orca/lib/bigdl-orca-spark_2.4.6-0.14.0-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 
2022-02-10 21:42:16 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-02-10 21:42:20,290 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-02-10 21:42:20,294 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-02-10 21:42:20,295 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-02-10 21:42:20,296 Thread-4 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-02-10 21:42:20 [Thread-4] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-02-10 21:42:20 [Thread-4] INFO  Engine$:123 - Executor number is 1 and executor cores number is 1
22-02-10 21:42:21 [Thread-4] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 15
2022-02-10 21:42:21 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.
22-02-10 21:42:21 [Thread-4] INFO  Engine$:446 - Find existing spark context. Checking the spark conf...
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.Sample
BigDLBasePickler registe


User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_MWAIT_HINTS=0
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_

**POSSIBLE 5-NETWORK STACKING**

```
# Define the network
class IntegratedNet(nn.Module):
  def __init__(self):
    super(IntegratedNet, self).__init__()

    self.resnet18 = models.resnet18(pretrained=True)
    self.resnet18.fc = nn.Linear(512, 32)

    self.densenet = models.densenet161(pretrained=True)
    self.densenet.classifier = nn.Linear(2208, 32)

    self.inception = models.inception_v3(pretrained=True)
    self.inception.fc = nn.Linear(2048, 32)
    
    self.mnasnet = models.mnasnet1_0(pretrained=True)
    self.mnasnet.classifier = nn.Sequential(nn.Dropout(0.2, inplace=True),
                                            nn.Linear(1280, 32))

    self.mobilenet_v2 = models.mobilenet_v2(pretrained=True)
    self.mobilenet_v2.classifier = nn.Sequential(nn.Dropout(0.2), nn.Linear(1280, 32))
    

    self.fc_out = nn.Linear(2*32, 1)  # for binary classification, use single output

  def forward(self, x):
    x_res = self.resnet18(x)
    x_dense = self.densenet(x.detach())
    x_inception = self.inception(x)[0]
    x_mnas = self.mnasnet(x)
    x_mobilenet = self.mobilenet_v2(x)
    
    #Concatenate the outputs
    x = torch.cat((x_res, x_dense, x_inception, x_mnas, x_mobilenet), axis=-1)
    x = self.fc_out(x)
    return x



```

In [ ]:
# Check that stacking works as expected
a = torch.rand(size=(8, 32))
b = torch.rand(size=(8, 32))
c = torch.rand(size=(8, 32))
print(a.shape)
d = torch.cat((a, b, c), axis=-1)
print(d.shape)

torch.Size([8, 32])
torch.Size([8, 96])


The stacking works well and we can concatenate the models if we have the same shape. That means that we will need to look at every last layer of each network to be sure they match. The networks themselves can be improved even further using [industry-standard best practices for transfered learning](https://arxiv.org/pdf/1911.02685.pdf)

In [ ]:
# Define the network
class IntegratedNet(nn.Module):
  def __init__(self):
    super(IntegratedNet, self).__init__()

    self.resnet18 = models.resnet18(pretrained=True)
    for param in self.resnet18.parameters():
      param.requires_grad = False
    self.resnet18.avgpool = nn.Sequential(nn.ReLU(),
                          nn.MaxPool2d(2, 2),
                          nn.Conv2d(in_channels=512, out_channels=64, kernel_size=3, stride=1, padding=1),
                          nn.BatchNorm2d(num_features=64),
                          nn.ReLU(),
                          nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1),
                          nn.BatchNorm2d(num_features=32),
                          nn.ReLU(),
                          nn.AdaptiveAvgPool2d(output_size=1))
    self.resnet18.fc = nn.Identity()
    # This code will help us to mantain the weights and extract the features according to the papers research to improve the 
    # performance of the transfering leaning (https://arxiv.org/pdf/1911.02685.pdf)
    self.mnasnet = models.mnasnet1_0(pretrained=True)
    for param in self.mnasnet.parameters():
      param.requires_grad = False
    self.mnasnet.classifier = nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                                      nn.ReLU(),
                                      nn.Linear(in_features=1280, out_features=256),
                                      nn.Dropout(p=0.2, inplace=True),
                                      nn.ReLU(),
                                      nn.Linear(256, 32))
    

    self.fc_out = nn.Sequential(nn.Linear(2*32, 1), nn.Sigmoid()) 
    # For binary classification, use single output
    # which will return values between 0 and 1 

  def forward(self, x):
    x_res = self.resnet18(x)
    x_mnas = self.mnasnet(x)
    x = torch.cat((x_res, x_mnas), axis=-1)
    x = self.fc_out(x)
    return x

In [ ]:
net = IntegratedNet()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
# We define the training loss vs. the epochs
criterion = nn.BCELoss() # and the binary cross entropy 
batch_size = 32
epochs = 2

In [ ]:
# We can (finally) proceed to import the Orca stimators and metrics.
# Since our output is binary (1: pneumonia + ; 0: pneumonia -) we need 
# a binary accuracy metric
from bigdl.orca.learn.pytorch import Estimator 
from bigdl.orca.learn.metrics import BinaryAccuracy  

est = Estimator.from_torch(model=net, optimizer=optimizer, loss=criterion, metrics=[BinaryAccuracy()])

creating: createTorchLoss
creating: createTorchOptim
creating: createZooKerasBinaryAccuracy
creating: createEstimator


  **TRAINING THE MACHINE**

In [ ]:
# Each epoch, we will try to fit the data
from bigdl.orca.learn.trigger import EveryEpoch 

est.fit(data=train_loader, epochs=2, validation_data=test_loader,
        checkpoint_trigger=EveryEpoch())

creating: createEveryEpoch
creating: createMaxEpoch


22-02-10 21:42:56 [Thread-4] INFO  InternalDistriOptimizer$:944 - TorchModel[f33d891a] isTorch is true
22-02-10 21:42:56 [Thread-4] INFO  InternalDistriOptimizer$:950 - torch model will use 1 OMP threads.
22-02-10 21:42:56 [Thread-4] INFO  DistriOptimizer$:826 - caching training rdd ...


22-02-10 21:43:26 [Thread-4] INFO  DistriOptimizer$:652 - Cache thread models...
22-02-10 21:43:26 [Executor task launch worker for task 5] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 51
22-02-10 21:43:26 [Executor task launch worker for task 5] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 51
22-02-10 21:43:26 [Executor task launch worker for task 5] INFO  DistriOptimizer$:635 - model thread pool size is 1
2022-02-10 21:43:26 WARN  BlockManager:66 - Asked to remove block test_0weights0, which does not exist
2022-02-10 21:43:26 WARN  BlockManager:66 - Asked to remove block test_0gradients0, which does not exist
22-02-10 21:43:26 [Thread-4] INFO  DistriOptimizer$:654 - Cache thread models... done
22-02-10 21:43:26 [Thread-4] INFO  DistriOptimizer$:164 - Count dataset


22-02-10 21:44:35 [Thread-4] INFO  DistriOptimizer$:168 - Count dataset complete. Time elapsed: 68.657940791s


22-02-10 21:45:43 [Thread-4] WARN  DistriOptimizer$:170 - If the dataset is built directly from RDD[Minibatch], the data in each minibatch is fixed, and a single minibatch is randomly selected in each partition. If the dataset is transformed from RDD[Sample], each minibatch will be constructed on the fly from random samples, which is better for convergence.
22-02-10 21:45:43 [Thread-4] INFO  DistriOptimizer$:176 - config  {
	computeThresholdbatchSize: 100
	maxDropPercentage: 0.0
	warmupIterationNum: 200
	isLayerwiseScaled: false
	dropPercentage: 0.0
 }
22-02-10 21:45:43 [Thread-4] INFO  DistriOptimizer$:180 - Shuffle data
22-02-10 21:45:43 [Thread-4] INFO  DistriOptimizer$:183 - Shuffle data complete. Takes 1.2321E-4s


...

22-02-10 22:24:08 [Thread-4] INFO  DistriOptimizer$:430 - [Epoch 2 5120/5216][Iteration 323][Wall Clock 2204.152217718s] Trained 32.0 records in 6.682365607 seconds. Throughput is 4.7887235 records/second. Loss is 0.011642132. 


22-02-10 22:24:15 [Thread-4] INFO  DistriOptimizer$:430 - [Epoch 2 5152/5216][Iteration 324][Wall Clock 2210.818264148s] Trained 32.0 records in 6.66604643 seconds. Throughput is 4.8004465 records/second. Loss is 0.035903994. 


22-02-10 22:24:22 [Thread-4] INFO  DistriOptimizer$:430 - [Epoch 2 5184/5216][Iteration 325][Wall Clock 2217.585153504s] Trained 32.0 records in 6.766889356 seconds. Throughput is 4.728908 records/second. Loss is 0.02853406. 


22-02-10 22:24:29 [Thread-4] INFO  DistriOptimizer$:430 - [Epoch 2 5216/5216][Iteration 326][Wall Clock 2224.273770891s] Trained 32.0 records in 6.688617387 seconds. Throughput is 4.7842474 records/second. Loss is 0.04148032. 
22-02-10 22:24:29 [Thread-4] INFO  DistriOptimizer$:475 - [Epoch 2 5216/5216][Iteration 326][Wall Clock 2224.273770891s] Epoch finished. Wall clock time is 2325974.391781 ms
22-02-10 22:24:29 [Thread-4] INFO  DistriOptimizer$:112 - [Epoch 2 5216/5216][Iteration 326][Wall Clock 2224.273770891s] Validate model...


22-02-10 22:26:10 [Thread-4] INFO  DistriOptimizer$:178 - [Epoch 2 5216/5216][Iteration 326][Wall Clock 2224.273770891s] validate model throughput is 6.124821 records/second
22-02-10 22:26:10 [Thread-4] INFO  DistriOptimizer$:181 - [Epoch 2 5216/5216][Iteration 326][Wall Clock 2224.273770891s] Top1Accuracy is Accuracy(correct: 419, count: 624, accuracy: 0.6714743589743589)


In [ ]:
result = est.evaluate(data=test_loader)
for r in result:
    print(r, ":", result[r])

[Stage 665:>                                                        (0 + 1) / 1]

22-02-10 22:27:53 [Thread-4] INFO  DistriOptimizer$:1518 - Top1Accuracy is Accuracy(correct: 426, count: 624, accuracy: 0.6826923076923077)
Top1Accuracy : 0.682692289352417


The final accuracy is 68.26%.

In [ ]:
est.save("/content/drive/MyDrive/Colab_Notebooks/integratedNet") 
# We will finally save the model

'/content/drive/MyDrive/Colab_Notebooks/integratedNet'

In [ ]:
# And stop the orca context when program finishes
stop_orca_context()

Stopping orca context
